# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [1]:
import sklearn

In [2]:
sklearn.__version__

'0.22.1'

In [7]:
# import libraries
import pandas as pd
from sqlalchemy import create_engine

In [8]:
# load data from database
engine = create_engine('sqlite:///data/data.sqlite')
df = pd.read_sql("SELECT * FROM message_response", con=engine)
df.shape

(26207, 38)

In [9]:
df.head()

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,Weather update - a cold front from Cuba that c...,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Is the Hurricane over or is it not over,direct,1,0,0,1,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
2,Looking for someone but no name,direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,"says: west side of Haiti, rest of the country ...",direct,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
df.isnull().any().sum()

0

In [12]:
df[df['related'].isnull()]

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report


In [13]:
import sklearn

In [14]:
sklearn.__version__

'0.22.1'

In [15]:
df.shape

(26207, 38)

In [16]:
df.sample(5)

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
15424,An emergency conference call today with fellow...,news,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3885,The people in Fontamara have a lot of problems...,direct,1,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
16503,The weather forecasting centre said the heat w...,news,1,0,0,0,0,0,0,0,...,0,0,1,1,0,0,0,0,0,0
11160,"53 matchbooks , several flashlights , a few ba...",direct,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
1138,I want to know if I am a victim and do not hav...,direct,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [17]:
X = df['message'].values
Y = df.drop(['message', 'genre'], axis=1).copy().values

In [18]:
Y.shape

(26207, 36)

In [19]:
X.shape

(26207,)

### 2. Write a tokenization function to process your text data

In [20]:
from nltk.tokenize import word_tokenize

In [21]:
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

[nltk_data] Downloading package punkt to /home/uno/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/uno/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/uno/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [22]:
from nltk.stem import WordNetLemmatizer

In [23]:
import re

In [24]:
def tokenize(text):
#     text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
    
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    return clean_tokens

In [25]:
tokenize("this is . very message, that.. I have no idea is about. and suddently I have to think, about it")

['this',
 'is',
 '.',
 'very',
 'message',
 ',',
 'that',
 '..',
 'i',
 'have',
 'no',
 'idea',
 'is',
 'about',
 '.',
 'and',
 'suddently',
 'i',
 'have',
 'to',
 'think',
 ',',
 'about',
 'it']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [26]:
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier

In [27]:
from sklearn.ensemble import RandomForestClassifier

In [28]:
from sklearn.pipeline import Pipeline, FeatureUnion

In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

In [30]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier(n_jobs=-1)))
])

In [28]:
pipeline = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier(n_jobs=-1)))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [31]:
from sklearn.model_selection import train_test_split

In [32]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.3, random_state=2020)

In [33]:
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
         steps=[('vect',
                 CountVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.int64'>, encoding='utf-8',
                                 input='content', lowercase=True, max_df=1.0,
                                 max_features=None, min_df=1,
                                 ngram_range=(1, 1), preprocessor=None,
                                 stop_words=None, strip_accents=None,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenize at 0x7f4615dc4488>,
                                 vocabulary=None)),
                ('tfidf',
                 TfidfTransformer(norm='l2', smooth_idf=True,
                                  sublinear_tf=False, use_idf=True)),
                ('clf',
                 MultiOutputClassifier(estimator=KNeighborsClassifier(algorithm='auto',

In [34]:
Y_predict = pipeline.predict(X_test)

In [37]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [40]:
# pipeline.score(X_test, Y_test)

0.25670692943420215

In [ ]:
# pipeline.score(X_train, Y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [35]:
df.columns

Index(['message', 'genre', 'related', 'request', 'offer', 'aid_related',
       'medical_help', 'medical_products', 'search_and_rescue', 'security',
       'military', 'child_alone', 'water', 'food', 'shelter', 'clothing',
       'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report'],
      dtype='object')

In [36]:
names = ['related', 'request', 'offer', 'aid_related',
       'medical_help', 'medical_products', 'search_and_rescue', 'security',
       'military', 'child_alone', 'water', 'food', 'shelter', 'clothing',
       'money', 'missing_people', 'refugees', 'death', 'other_aid',
       'infrastructure_related', 'transport', 'buildings', 'electricity',
       'tools', 'hospitals', 'shops', 'aid_centers', 'other_infrastructure',
       'weather_related', 'floods', 'storm', 'fire', 'earthquake', 'cold',
       'other_weather', 'direct_report']

In [37]:
len(Y_predict[0])

36

In [38]:
from sklearn.metrics import classification_report

In [39]:
len(Y_predict[:, 0])

7863

In [40]:
import warnings

In [41]:
warnings.filterwarnings('ignore')

In [43]:
df['related'].value_counts()

1    19914
0     6101
2      192
Name: related, dtype: int64

In [45]:
df[df['related'] == 2]

,message,genre,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
117,Dans la zone de Saint Etienne la route de Jacm...,direct,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
219,"Hygienic . .. , mobile toilets, batteries, gen...",direct,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
305,I am a surviver of the earthkaque see how you ...,direct,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
460,I am located in Route Freres- Impasse Fortin. ...,direct,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
576,"Please people in Carrefour, Magloire Ambroide ...",direct,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20317,He said the government has provided a motorise...,news,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
20488,The Army has deployed a total of 72 columns in...,news,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
22317,"With a CERF allocation of almost $1.7 million,...",news,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
23373,"In this scenario, total housing financing need...",news,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [42]:
for i in range(len(Y_predict[0])):
    print(names[i])
    print(classification_report(Y_predict[:, i], Y_test[:, i]))
    

related
              precision    recall  f1-score   support

           0       0.21      0.29      0.25      1306
           1       0.85      0.77      0.81      6550
           2       0.02      0.14      0.03         7

    accuracy                           0.69      7863
   macro avg       0.36      0.40      0.36      7863
weighted avg       0.74      0.69      0.71      7863

request
              precision    recall  f1-score   support

           0       0.98      0.83      0.90      7655
           1       0.06      0.39      0.11       208

    accuracy                           0.82      7863
   macro avg       0.52      0.61      0.50      7863
weighted avg       0.96      0.82      0.88      7863

offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      7863
           1       0.00      0.00      0.00         0

    accuracy                           1.00      7863
   macro avg       0.50      0.50      0.50      786

              precision    recall  f1-score   support

           0       1.00      0.99      0.99      7863
           1       0.00      0.00      0.00         0

    accuracy                           0.99      7863
   macro avg       0.50      0.49      0.50      7863
weighted avg       1.00      0.99      0.99      7863

other_infrastructure
              precision    recall  f1-score   support

           0       1.00      0.96      0.98      7863
           1       0.00      0.00      0.00         0

    accuracy                           0.96      7863
   macro avg       0.50      0.48      0.49      7863
weighted avg       1.00      0.96      0.98      7863

weather_related
              precision    recall  f1-score   support

           0       0.84      0.76      0.80      6270
           1       0.31      0.43      0.36      1593

    accuracy                           0.69      7863
   macro avg       0.57      0.59      0.58      7863
weighted avg       0.73      0.69    

In [47]:
import joblib

In [48]:
joblib.dump(pipeline, 'models/multi_clf.pkl')

['models/multi_clf.pkl']

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.